In [58]:
import pandas as pd
import numpy as np
#https://machinelearningmastery.com/divergence-between-probability-distributions/
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import sqlalchemy
from sklearn.preprocessing import StandardScaler
df_stikhi = pd.read_csv('./100520_stikhi.csv', index_col = 0)

df_roman = pd.read_csv('./101620_roman.csv', index_col = 0)
df_manifest = pd.read_csv('./082820_manifestessay.csv', index_col = 0)
df_dnevnik = pd.read_csv('./072420_dnevnikcats.csv', index_col = 0)

In [68]:
df_dnevnik = df_dnevnik[['diarynum', 'entry_year', 'entry_id', 'X0', 'X1', 'X2']]
df_dnevnik = df_dnevnik.rename(columns = {'X0': '0', 'X1': '1', 'X2': '2'})

In [48]:
def to_dims(df_stikhi):
    scaler = StandardScaler()
    df_stikhi['0'] = scaler.fit_transform(df_stikhi[['0']])

    df_stikhi['1'] = scaler.fit_transform(df_stikhi[['1']])

    df_stikhi['2'] = scaler.fit_transform(df_stikhi[['2']])

    df_stikhi['0_pos'] = [i if i > 0 else 0 for i in df_stikhi['0']]
    df_stikhi['0_neg'] = [abs(i) if i < 0 else 0 for i in df_stikhi['0']]


    df_stikhi['1_pos'] = [i if i > 0 else 0 for i in df_stikhi['1']]
    df_stikhi['1_neg'] = [abs(i) if i < 0 else 0 for i in df_stikhi['1']]


    df_stikhi['2_pos'] = [i if i > 0 else 0 for i in df_stikhi['2']]
    df_stikhi['2_neg'] = [abs(i) if i < 0 else 0 for i in df_stikhi['2']]

    return(df_stikhi)

In [49]:

def splityear (df, year_div = 1950):
    return([df[df['entry_year'] >= year_div], df[df['entry_year'] < year_div]])

def to_bar (df):
    hm = pd.melt(df)
    hm = hm[hm['value'] > 0]
    hm = hm.groupby(['variable']).sum()
    hm = hm.reset_index(drop = False)
    hm['percent'] =  hm['value']/hm['value'].sum()

    dim_1 = hm['variable']
    dim2_1 = hm['percent']
    return([dim_1, dim2_1])

In [55]:
def ready_for_bar(df_stikhi):
    df_stikhi = to_dims(df_stikhi)

    new_stikhi = df_stikhi.iloc[:, 6:]
    new_stikhi['entry_year'] = df_stikhi['entry_year']

    stikhi = splityear(new_stikhi)
    stikhi_1 = stikhi[1].drop('entry_year', axis = 1)
    stikhi_2 = stikhi[0].drop('entry_year', axis = 1)
    
    return(stikhi_1, stikhi_2)

In [71]:
stikhi_1, stikhi_2 = ready_for_bar(df_stikhi)
roman_1, roman_2 = ready_for_bar(df_roman)
manifest_1, manifest_2 = ready_for_bar(df_manifest)
dnevnik_1, dnevnik_2 = ready_for_bar(df_dnevnik)

In [72]:
fig = make_subplots(rows=2, cols=2,
          subplot_titles=("Poetry","Novels", "Manifesto", "Diaries"))

dims1 = to_bar(stikhi_1)
fig.add_trace(go.Bar(x=dims1[0], y = dims1[1], name = 'pre 1950'), row = 1, col = 1)
dims2 = to_bar(stikhi_2)
fig.add_trace(go.Bar(x=dims2[0], y = dims2[1], name = 'post 1950'), row = 1, col = 1)

dims3 = to_bar(roman_1)
fig.add_trace(go.Bar(x=dims3[0], y = dims3[1], name = 'pre 1950'), row = 1, col = 2)
dims4 = to_bar(roman_2)
fig.add_trace(go.Bar(x=dims4[0], y = dims4[1], name = 'post 1950'), row = 1, col = 2)

dims5 = to_bar(manifest_1)
fig.add_trace(go.Bar(x=dims5[0], y = dims5[1], name = 'pre 1950'), row = 2, col = 1)
dims6 = to_bar(manifest_2)
fig.add_trace(go.Bar(x=dims6[0], y = dims6[1], name = 'post 1950'), row = 2, col = 1)


dims7 = to_bar(dnevnik_1)
fig.add_trace(go.Bar(x=dims7[0], y = dims7[1], name = 'pre 1950'), row = 2, col = 2)
dims8 = to_bar(dnevnik_2)
fig.add_trace(go.Bar(x=dims8[0], y = dims8[1], name = 'post 1950'), row = 2, col = 2)


In [73]:
from scipy.spatial.distance import jensenshannon as jsdis
js = jsdis(dims1[1], dims2[1], base=2)
js

0.02793880195581659

In [74]:
js = jsdis(dims3[1], dims4[1], base=2)
js

0.06063069295280717

In [75]:
js = jsdis(dims5[1], dims6[1], base=2)
js

0.4104981245768497

In [76]:
js = jsdis(dims7[1], dims8[1], base=2)
js

0.05136866108659136